In [1]:
import nltk
import pandas as pd
import numpy as np
import tensorflow.keras

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,precision_score,recall_score,f1_score

In [3]:
text=pd.read_csv("datasets/plain.txt",sep='\n',skiprows=2,header=None)
text.head()
text.rename(columns={0:'txt'},inplace=True)
text2=pd.read_csv("datasets/sql_querys.txt",sep='\n',header=None)

In [4]:
s1=list(text['txt'])
s2=list(text2[0])
s1=[i.lower() for i in s1]
s2=[i.lower() for i in s2]

In [5]:
y1=[0 for i in range(len(s1))]
y2=[1 for i in range(len(s2))]
y=y1+y2
print(f"Total plain text: {len(s1)}")
print(f"Total sql injection queries: {len(s2)}")

Total plain text: 3692
Total sql injection queries: 5928


In [6]:
word_lem=WordNetLemmatizer()

In [7]:
new_s1=[]
stopwords=nltk.corpus.stopwords.words("english")
for i in s1:
    temp=i.split(' ')
    temp1=[]
    for j in temp:

        j=j.strip(',').strip(r'["]').strip(';').strip('“').strip('”').strip(r'[.]+')
        j=j.replace("’",'')
        if j not in stopwords and j.isalpha():
            
            j=word_lem.lemmatize(j)
            temp1.append(j)

    new_s1.append(' '.join(temp1))

In [8]:
new_s2=[]
for i in s2:
    temp=i.split(' ')
    temp1=[]
    for j in temp:
        if j not in stopwords:
            
            j=word_lem.lemmatize(j)
            temp1.append(j)

    new_s2.append(' '.join(temp1))

In [11]:
new_s = new_s1+new_s2
tokenizer1 = Tokenizer(num_words = 100000, oov_token="<OOV>")
tokenizer1.fit_on_texts(new_s)
word_index1 = tokenizer1.word_index

In [12]:
sequences1 = tokenizer1.texts_to_sequences(new_s)
n = 0
for i in sequences1:
    n = max(n,len(i))

In [13]:
padded1 = pad_sequences(sequences1, maxlen=150,padding="post")

In [14]:
df_new=pd.DataFrame(padded1)
df_new["target"]=y
df_new.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,target
0,260,950,1874,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4062,1875,1603,260,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4063,392,6984,4064,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,844,1094,606,951,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,760,2933,370,2934,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X_train,X_test,y_train,y_test=train_test_split(df_new.iloc[:,:150],pd.Series(y),random_state=42)

In [16]:
model=tensorflow.keras.Sequential()
model.add(tensorflow.keras.layers.Embedding(17000,16))
model.add(tensorflow.keras.layers.GlobalAveragePooling1D())
model.add(tensorflow.keras.layers.Dense(16,activation="relu"))
model.add(tensorflow.keras.layers.Dropout(rate=0.3))
model.add(tensorflow.keras.layers.Dense(32,activation="relu"))
model.add(tensorflow.keras.layers.Dropout(rate=0.3))
model.add(tensorflow.keras.layers.Dense(1,activation='sigmoid'))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [17]:
model.fit(np.array(X_train),np.array(y_train),epochs=10,batch_size=50,verbose=1)

Epoch 1/10
145/145 [==============================] - 1s 6ms/step - loss: 0.6567 - accuracy: 0.6137
Epoch 2/10
145/145 [==============================] - 1s 6ms/step - loss: 0.4552 - accuracy: 0.7792
Epoch 3/10
145/145 [==============================] - 1s 5ms/step - loss: 0.2208 - accuracy: 0.9454: 0s - loss: 0.2285 - accuracy: 0.
Epoch 4/10
145/145 [==============================] - 1s 5ms/step - loss: 0.1136 - accuracy: 0.9708
Epoch 5/10
145/145 [==============================] - 1s 6ms/step - loss: 0.0685 - accuracy: 0.9803
Epoch 6/10
145/145 [==============================] - 1s 5ms/step - loss: 0.0539 - accuracy: 0.9836
Epoch 7/10
145/145 [==============================] - 1s 5ms/step - loss: 0.0387 - accuracy: 0.9878
Epoch 8/10
145/145 [==============================] - 1s 6ms/step - loss: 0.0276 - accuracy: 0.9931
Epoch 9/10
145/145 [==============================] - 1s 6ms/step - loss: 0.0218 - accuracy: 0.9951
Epoch 10/10
145/145 [==============================] - 1s 5ms/step

In [18]:
def build_classifier():
    model=tensorflow.keras.Sequential()
    model.add(tensorflow.keras.layers.Embedding(17000,16))
    model.add(tensorflow.keras.layers.GlobalAveragePooling1D())
    model.add(tensorflow.keras.layers.Dense(16,activation="relu"))
    model.add(tensorflow.keras.layers.Dropout(rate=0.3))
    model.add(tensorflow.keras.layers.Dense(32,activation="relu"))
    model.add(tensorflow.keras.layers.Dropout(rate=0.3))
    model.add(tensorflow.keras.layers.Dense(1,activation='sigmoid'))

    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    
    return model

In [19]:
classifier=KerasClassifier(build_fn=build_classifier,batch_size=50,epochs=10)
accuracies=cross_val_score(estimator=classifier,X=X_train,y=y_train,cv=10)

Epoch 1/10
130/130 [==============================] - 1s 5ms/step - loss: 0.6716 - accuracy: 0.6171
Epoch 2/10
130/130 [==============================] - 1s 5ms/step - loss: 0.5705 - accuracy: 0.6187
Epoch 3/10
130/130 [==============================] - 1s 5ms/step - loss: 0.3788 - accuracy: 0.8195
Epoch 4/10
130/130 [==============================] - 1s 5ms/step - loss: 0.2980 - accuracy: 0.9501
Epoch 5/10
130/130 [==============================] - 1s 5ms/step - loss: 0.2465 - accuracy: 0.9797
Epoch 6/10
130/130 [==============================] - 1s 5ms/step - loss: 0.2100 - accuracy: 0.9926
Epoch 7/10
130/130 [==============================] - 1s 6ms/step - loss: 0.1847 - accuracy: 0.9941
Epoch 8/10
130/130 [==============================] - 1s 5ms/step - loss: 0.1642 - accuracy: 0.9952
Epoch 9/10
130/130 [==============================] - 1s 6ms/step - loss: 0.1448 - accuracy: 0.9989
Epoch 10/10
15/15 [==============================] - 0s 1ms/step - loss: 0.1541 - accuracy: 0.9751
E

130/130 [==============================] - 1s 5ms/step - loss: 0.0585 - accuracy: 0.9828
Epoch 7/10
130/130 [==============================] - 1s 5ms/step - loss: 0.0439 - accuracy: 0.9877
Epoch 8/10
130/130 [==============================] - 1s 5ms/step - loss: 0.0349 - accuracy: 0.9892
Epoch 9/10
130/130 [==============================] - 1s 5ms/step - loss: 0.0303 - accuracy: 0.9917: 0s - loss: 0.0326 - accuracy
Epoch 10/10
15/15 [==============================] - 0s 1ms/step - loss: 0.0464 - accuracy: 0.9778
Epoch 1/10
130/130 [==============================] - 1s 5ms/step - loss: 0.6700 - accuracy: 0.6087
Epoch 2/10
130/130 [==============================] - 1s 5ms/step - loss: 0.5665 - accuracy: 0.6551
Epoch 3/10
130/130 [==============================] - 1s 6ms/step - loss: 0.3440 - accuracy: 0.8954
Epoch 4/10
130/130 [==============================] - 1s 5ms/step - loss: 0.1968 - accuracy: 0.9617
Epoch 5/10
130/130 [==============================] - 1s 9ms/step - loss: 0.1300 -

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          272000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 3

In [22]:
score=model.evaluate(X_test,y_test)
print(score)

76/76 [==============================] - 0s 1ms/step - loss: 0.0615 - accuracy: 0.9734
[0.06146598234772682, 0.9733887910842896]


In [23]:
model.save("model/model.h5")